In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize
from numpy.linalg import inv,norm
from scipy.special import beta,betainc
from sklearn.linear_model import LassoLars, lars_path

In [2]:
# Defining a file path and loading excel file
xls_file = pd.ExcelFile('Data_Project_2.xlsx')
e_worksheet_name = xls_file.sheet_names

In [3]:
# Extract all worksheets from excel and assign to new variable dataframes
famafrench = xls_file.parse(e_worksheet_name[0])
mutualfund = xls_file.parse(e_worksheet_name[1])
smartbeta = xls_file.parse(e_worksheet_name[2])
hedgefund = xls_file.parse(e_worksheet_name[3])

In [4]:
famafrench = famafrench[(famafrench['Date']>=200101) & (famafrench['Date']<=201912)]

In [5]:
# Reset index after selecting data based on dates for all worksheets.
# This is done so that we can run regression based on same Df length.

famafrench = famafrench.reset_index(drop=True)
famafrench

,Date,mktrf,smb,hml,RMW,CMA,rf,umd
0,200101,3.13,5.78,-5.28,-5.31,-6.55,0.54,-25.05
1,200102,-10.05,2.63,12.60,9.09,9.56,0.38,12.51
2,200103,-7.26,2.27,6.52,3.27,3.94,0.42,8.35
3,200104,7.94,-0.67,-4.74,-3.42,-3.95,0.39,-7.97
4,200105,0.72,3.55,3.38,0.17,2.18,0.32,2.12
...,...,...,...,...,...,...,...,...
223,201908,-2.58,-3.30,-4.93,0.43,-0.88,0.16,7.66
224,201909,1.43,0.26,6.78,1.99,3.50,0.18,-6.85
225,201910,2.06,0.21,-2.09,0.25,-0.99,0.15,0.29
226,201911,3.87,0.50,-1.87,-1.58,-1.24,0.12,-2.60


In [6]:
mutualfund = mutualfund.iloc[12:]
# Assuming the end date of each worksheet is the same end date as 
# Famafrench worksheet, we will copy the columns 'Date' into the other 
# worksheets. As mentioned in the previous cell, we will reset the index
# before copying columns.
mutualfund = mutualfund.reset_index(drop=True)
mutualfund['Date'] = famafrench['Date']
mutualfund_excess = pd.DataFrame()
for i in mutualfund.iloc[:,1:]:
    mutualfund_excess[i] = (mutualfund[i]-(famafrench['rf']/100))
mutualfund_excess['Date'] = mutualfund['Date']
mutualfund_excess

,FSMEX,FSELX,INPIX,SLMCX,CSEIX,FSPHX,NASDX,JAENX,JENSX,DAPAX,Date
0,-0.027630,0.408666,0.108570,0.238078,0.041540,-0.008050,0.098904,0.045285,-0.036892,0.131434,200101
1,0.016364,-0.297978,-0.508751,-0.205961,-0.014710,-0.017997,-0.267668,-0.221774,-0.025203,-0.136422,200102
2,-0.090673,-0.075910,-0.397533,-0.073921,0.004991,-0.088075,-0.183426,-0.184689,-0.061902,-0.090705,200103
3,0.012327,0.253523,0.334561,0.235072,0.042474,0.025668,0.174760,0.142741,0.035611,0.083330,200104
4,0.065974,-0.074223,-0.024136,-0.025668,0.016253,0.025640,-0.032674,-0.014627,0.021063,-0.020552,200105
...,...,...,...,...,...,...,...,...,...,...,...
223,-0.013437,-0.038220,-0.095215,-0.030842,0.047591,-0.028800,-0.022455,-0.018193,-0.012694,-0.012344,201908
224,-0.016282,0.043034,-0.033417,0.019207,0.002948,-0.026060,0.004057,0.008382,0.010205,-0.004127,201909
225,0.019545,0.062866,-0.014216,0.044437,0.021359,0.065504,0.043998,-0.003588,0.015058,0.024939,201910
226,0.061344,0.055768,0.041037,0.051132,-0.037205,0.088847,-0.027088,0.038408,0.034090,0.045012,201911


In [7]:
smartbeta = smartbeta.reset_index(drop=True)
smartbeta['Date'] = famafrench['Date']
smartbeta_excess = pd.DataFrame()
for i in smartbeta.iloc[:,1:]:
    smartbeta_excess[i] = (smartbeta[i]-(famafrench['rf']/100))
smartbeta_excess['Date'] = smartbeta['Date']

In [8]:
hedgefund = hedgefund[(hedgefund['Date']>=200101) & (hedgefund['Date']<=201912)]
hedgefund.reset_index(drop=True,inplace=True)
hedgefund['Date'] = famafrench['Date']
hedgefund_excess = pd.DataFrame()
for i in hedgefund.iloc[:,1:]:
    hedgefund_excess[i] = (hedgefund[i]-(famafrench['rf']/100))
hedgefund_excess['Date'] = hedgefund['Date']

In [9]:
# Define a function to test for alpha under FF5+MOM
def FF5MOM_test(df1):
    LIST=[]
    
    # Use a for loop to parse through all the columns of each df
    for column in df1.iloc[:,1:]:
        returndf1 = df1[column].copy()
        
        # to account for nan values, we fill them with the mean of the column
        returndf1.fillna(returndf1.mean(),inplace=True)
        model_famafrench = sm.OLS(returndf1, sm.add_constant(famafrench.iloc[:,[1,2,3,4,5,7]])).fit()
        model = model_famafrench.summary()
        LIST.append(model)
    return LIST

In [10]:
FF5MOM_test(mutualfund_excess)

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                  FSELX   R-squared:                       0.731
 Model:                            OLS   Adj. R-squared:                  0.723
 Method:                 Least Squares   F-statistic:                     99.87
 Date:                Sun, 22 Oct 2023   Prob (F-statistic):           3.93e-60
 Time:                        00:41:37   Log-Likelihood:                 367.17
 No. Observations:                 228   AIC:                            -720.3
 Df Residuals:                     221   BIC:                            -696.3
 Df Model:                           6                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [11]:
FF5MOM_test(smartbeta_excess)

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                   DWAQ   R-squared:                       0.732
 Model:                            OLS   Adj. R-squared:                  0.725
 Method:                 Least Squares   F-statistic:                     100.6
 Date:                Sun, 22 Oct 2023   Prob (F-statistic):           2.17e-60
 Time:                        00:41:37   Log-Likelihood:                 518.99
 No. Observations:                 228   AIC:                            -1024.
 Df Residuals:                     221   BIC:                            -1000.
 Df Model:                           6                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

In [12]:
FF5MOM_test(hedgefund_excess)

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                HFRIMAI   R-squared:                       0.461
 Model:                            OLS   Adj. R-squared:                  0.447
 Method:                 Least Squares   F-statistic:                     31.56
 Date:                Sun, 22 Oct 2023   Prob (F-statistic):           2.73e-27
 Time:                        00:41:38   Log-Likelihood:                 834.93
 No. Observations:                 228   AIC:                            -1656.
 Df Residuals:                     221   BIC:                            -1632.
 Df Model:                           6                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

### Global Minimum Variance

In [13]:
# First we create dataframes based on the in sample period for each excel sheet
# Then we fill the NaN values with mean of the column

In [14]:
mutualfund_is = mutualfund_excess[(mutualfund_excess['Date']>=200101) & (mutualfund_excess['Date']<=201212)]
mutualfund_is.drop(columns='Date',inplace=True)
mutualfund_is = mutualfund_is.apply(lambda col: col.fillna(col.mean()),axis=0)

C:\Users\azzra\AppData\Local\Temp\ipykernel_16796\2542682244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mutualfund_is.drop(columns='Date',inplace=True)


In [15]:
smartbeta_is = smartbeta_excess[(smartbeta_excess['Date']>=200101) & (smartbeta_excess['Date']<=201212)]
smartbeta_is.drop(columns='Date',inplace=True)
smartbeta_is = smartbeta_is.apply(lambda col: col.fillna(col.mean()),axis=0)

C:\Users\azzra\AppData\Local\Temp\ipykernel_16796\1198732335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smartbeta_is.drop(columns='Date',inplace=True)


In [16]:
hedgefund_is = hedgefund_excess[(hedgefund_excess['Date']>=200101) & (hedgefund_excess['Date']<=201212)]
hedgefund_is.drop(columns='Date',inplace=True)
hedgefund_is = hedgefund_is.apply(lambda col: col.fillna(col.mean()),axis=0)

C:\Users\azzra\AppData\Local\Temp\ipykernel_16796\1237126557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hedgefund_is.drop(columns='Date',inplace=True)


In [17]:
# Citation of code below

# Author: Danny Chen
# Portfolio-Optimization
# Code version 13
# Type: Source Code
# Web address: https://github.com/zhuodannychen/Portfolio-Optimization

In [18]:
# Define a function to calculate optimal weight for GMV

def minimum_variance(ret,long=True):
    def find_port_variance(weights):
# First we calculate the portfolio std dev using the code below
        cov = ret.cov()
        port_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights)) * 12)
        return port_std
    
# We then create a function that will be used in the constraint later
    def weight_cons(weights):
        return np.sum(weights) - 1

    # The bounds will be between 0 and 1 for the weights because there is no short selling
    if long == True:
        bounds_lim = [(0,1) for x in range(len(ret.columns))] # change to (-1, 1) if you want to short
    else:
        bounds_lim =[(-1,1) for x in range(len(ret.columns))]
    
    init = [1/len(ret.columns) for i in range(len(ret.columns))] # this is the guessed value for weights 
    constraint = {'type': 'eq', 'fun': weight_cons}

    optimal = minimize(fun=find_port_variance,
                       x0=init,
                       bounds=bounds_lim,
                       constraints=constraint,
                       method='SLSQP'
                       )

    return list(optimal['x'])

In [19]:
w_mutualfund_long = minimum_variance(mutualfund_is)
w_mutualfund_short = minimum_variance(mutualfund_is,False)

In [20]:
w_smartbeta_long = minimum_variance(smartbeta_is)
w_smartbeta_short = minimum_variance(smartbeta_is,False)

In [21]:
w_hedgefund_long = minimum_variance(hedgefund_is)
w_hedgefund_short = minimum_variance(hedgefund_is,False)

In [22]:
w_mutualfund_long

[0.29949151320234607,
 0.0,
 1.0462550964485118e-16,
 0.0,
 7.101524229780054e-17,
 0.0479558908138033,
 0.0,
 1.7564075194265172e-17,
 0.6525525959838507,
 0.0]

In [23]:
def GMV(ret, weight):
    mean_stat = []
    std_dev = []
    sharpe_r = []
    # use a for loop to make sure each asset weight is being multiplied correctly
    # to its respective return and covariance
    
    for i in range(len(weight)):
        mu = np.mean(ret.iloc[:, i]) # parse through each column and find its mean
        Sh = np.cov(ret, rowvar=False, ddof=0) # compute covariance matris
        mean_stat.append((mu * weight[i]*12)) # append the annualized portfolio return to a list
        std_dev.append(np.sqrt(12) * np.sqrt(np.dot(weight[i], Sh.dot(weight[i])))) # append annualized std dev to list
        sharpe_r.append(np.sqrt(12) * (np.dot(weight[i], mu) / std_dev[i])) # append annualized sharpe ratio to list

    return (np.mean(mean_stat), np.mean(std_dev), np.mean(sharpe_r))

In [24]:
print(GMV(mutualfund_is,w_mutualfund_long))
print(GMV(mutualfund_is,w_mutualfund_short))

(0.00406569772891765, 0.019857540223004472, nan)
(0.00246545421224417, 0.04907403607861935, -0.007847595321685684)


C:\Users\azzra\AppData\Local\Temp\ipykernel_16796\2518181319.py:13: RuntimeWarning: invalid value encountered in divide
  sharpe_r.append(np.sqrt(12) * (np.dot(weight[i], mu) / std_dev[i])) # append annualized sharpe ratio to list


In [25]:
print(GMV(smartbeta_is,w_smartbeta_long))
print(GMV(smartbeta_is,w_smartbeta_short))

(0.005676272781512471, 0.013320748644058302, nan)
(0.0014891836591054658, 0.04632236143846503, -0.003217697142383077)


C:\Users\azzra\AppData\Local\Temp\ipykernel_16796\2518181319.py:13: RuntimeWarning: invalid value encountered in divide
  sharpe_r.append(np.sqrt(12) * (np.dot(weight[i], mu) / std_dev[i])) # append annualized sharpe ratio to list


In [26]:
print(GMV(hedgefund_is,w_hedgefund_long))
print(GMV(hedgefund_is,w_hedgefund_short))

(0.001607941464113893, 0.005260694067345394, nan)
(0.0016207167064299355, 0.011412102857602665, -0.02884973117213247)


C:\Users\azzra\AppData\Local\Temp\ipykernel_16796\2518181319.py:13: RuntimeWarning: invalid value encountered in divide
  sharpe_r.append(np.sqrt(12) * (np.dot(weight[i], mu) / std_dev[i])) # append annualized sharpe ratio to list
